In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import json
#from sklearn.model_selection import train_test_split
#import torch
#from torch import nn, optim
import time

In [3]:
# Import connectome data

gapjn = {}
chemical = {}

# Gap Junction
gap_csv = pd.read_csv("gapjn.csv").fillna(0)
gap_data = gap_csv.values[:, 1:]
gap_labels = gap_csv.values[:, 0]

for i, label in enumerate(gap_labels):
    row_dict = {}
    for j, tag in enumerate(gap_labels):
        row_dict[tag] = gap_data[i, j]

    gapjn[label] = row_dict

# Chemical 
chem_csv = pd.read_csv("chemical.csv").fillna(0)
chem_data = chem_csv.values[:, 1:]
chem_labels = chem_csv.values[:, 0]
chem_cols = chem_csv.columns[1:]

for i, label in enumerate(chem_labels):
    row_dict = {}
    for j, tag in enumerate(chem_cols):
        row_dict[tag] = chem_data[i, j]

    chemical[label] = row_dict

In [4]:
# Import neuron data

with open("data/2022-08-02-01.json", "r") as file:
    data_text = file.read()
    data_dict = json.loads(data_text)
    neuron = np.array(data_dict["trace_array"])

In [12]:
with open("neuron_table.json", "r") as file:
    neurons = json.load(file)

## Model

### Neuron

In [53]:
def I_leak(V_m):
    E_leak = -35 #mV
    G_leak = 10 #pS I think

    return G_leak * (V_m - E_leak)

def delta_V_m(V_m, I_leak, I_gap, I_syn, I_in):
    C_m = 1 #pF

    current_sum = -I_leak - I_gap - I_syn + I_in
    return current_sum / C_m

# Simplified for now
def new_s(V_m):
    V_th = -20 #??

    return np.clip(V_m - V_th, 0, 1)

def I_gap(i, big_V, big_G_gap):
    sum = 0
    for j in range(len(big_V)):
        sum += big_G_gap[i, j] * (big_V[i] - big_V[j])

    return sum

def I_syn(i, big_V, big_E, big_s, big_G_syn):
    sum = 0
    for j in range(len(big_V)):
        sum += big_G_syn[i, j] * big_s[j] * (big_V[i] - big_E[j])
    return sum

In [61]:
def start_sim(time_step, run_time, input_amount=50, in_start=0.4, in_end=0.6):
    
    
    # Set inital values
    names = np.array(list(chemical.keys()))
    big_V = np.array([-40.0 for i in names])
    big_s = np.array([0.0 for i in names])
    big_G_gap = np.array([list(gapjn[name].values()) for name in names])
    big_G_syn = np.array([[gapjn[name][name_2] for name_2 in names] for name in names])
    big_E = np.array([0.0 for i in names])
    indices = np.array([i for i in range(len(names))])
    input_current = np.array([0.0 for i in names])
    
    t = 0

    # Storage
    big_V_array = []
    I_leak_array = []
    I_in_array = []
    I_syn_array = []
    time = []

    time_range = int(run_time / time_step)
    start_index = int(time_range * in_start)
    end_index = int(time_range * in_end)

    for i in range(time_range):
        if i == time_range // 4:
            print("25%")
        if i == time_range // 2:
            print("50%")
        if i == (time_range // 4) * 3:
            print("75%")

        if i == start_index:
            input_current[0] = input_amount
        
        if i == end_index:
            input_current[0] = 0

        # Time stuff
        time.append(t)
        t += time_step

        # Calculate
        leak_current = I_leak(big_V)
        
        gap_current = I_gap(indices, big_V, big_G_gap)
        syn_current = I_syn(indices, big_V, big_E, big_s, big_G_syn)
        
        d_V_m = delta_V_m(big_V, leak_current, gap_current, syn_current, input_current)
        d_s = new_s(big_V)

        #TEMP Voltage clamp
        d_V_m[0] = 0
        
        #Update
        big_V += d_V_m * time_step
        big_s += d_s * time_step
    
        #Store
        big_V_array.append(big_V.copy())
        I_leak_array.append(leak_current.copy())
        I_in_array.append(input_current.copy())
        I_syn_array.append(syn_current.copy())

    return [time, np.array(big_V_array) ,np.array(I_leak_array), np.array(I_in_array), np.array(I_syn_array)]

In [ ]:
results = start_sim(0.001, 3.0, in_start=1.2, in_end=1.2)

25%
50%
